# New baseline for the bovine with Transformers

inspired from this guide :
https://keras.io/examples/vision/video_transformers/

In [1]:
import matplotlib.pyplot as plt

In [2]:
from problem import get_train_data, get_test_data, WeightedClassificationError

In [3]:
import numpy as np
#CPU
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"  
import tensorflow as tf
from time import time
import pandas as pd
import math

In [4]:
from tensorflow.keras import layers
from tensorflow import keras

## Define hyperparams

In [5]:
#MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024 #nbr of feat output by densenet121
IMG_SIZE = 224
NB_FRAMES=30 #2.56gb vs 0.33gb

EPOCHS = 100


## Data preparation

In [6]:
videos_train, labels_train = get_train_data()

In [7]:
X_for_classifier= np.array(videos_train)
y_for_classifier= labels_train

In [8]:
videos_test, labels_test  = get_test_data()

In [22]:
Xtest_for_classifier = np.array(videos_test)
ytest_for_classifier = labels_test

In [14]:
from PIL import Image

def resize_frames(video, new_size):
    res=[]
    for frame in video:
        resized_img=Image.fromarray(frame).resize(size=(224, 224))
        res.append(np.array(resized_img))
    return np.array(res)

In [89]:
#function that gets all dataset
# 30 frames per video for 177 video = 2.65 gb !if considering each frame of float64
# as uint8 it takes 0.33 gb

def gen_videos(videolist):
    newvideos=[] # 177*30*250*250
    for video in videolist:
        reducedvideo= video.read_samples(video.frame_times[0:299:10])
        reducedvideo= reducedvideo.astype('uint8')
        #add dimnesion this takes quite a bit of memory ???? dim= 30*250*250*3
        reducedvideo=np.repeat(reducedvideo[...,np.newaxis], 3, -1)
        
        #CROP from 250 to 224()DenseNet121 standards !! TODO !
        reducedvideo=resize_frames(reducedvideo)
        #and add a batch dimension. dim= 1*30*250*250*3
        #reducedvideo = reducedvideo[None, ...]

        newvideos.append(reducedvideo)
    return newvideos

In [90]:
X_for_classifier= np.array(gen_videos(X_for_classifier))
X_for_classifier.shape

(177, 30, 224, 224, 3)

In [91]:
Xtest_for_classifier= np.array(gen_videos(Xtest_for_classifier))
Xtest_for_classifier.shape

(100, 30, 224, 224, 3)

In [92]:
def class_to_int(argument):
    switcher = {
        'A':0,
        'B':1,
        'C':2,
        'D':3,
        'E':4,
        'F':5,
        'G':6,
        'H':7,
    }
 
    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    return switcher.get(argument, "nothing")
 

In [93]:
func=np.vectorize(class_to_int)
#Train
train_labels=func(y_for_classifier)
#Test
test_labels=func(ytest_for_classifier)

In [94]:
train_labels

array([7, 7, 0, 5, 7, 0, 0, 5, 5, 0, 2, 1, 2, 5, 7, 5, 5, 7, 1, 1, 7, 2,
       7, 1, 1, 5, 7, 7, 7, 2, 2, 4, 6, 3, 4, 2, 0, 0, 7, 7, 6, 2, 5, 2,
       6, 1, 7, 7, 7, 2, 7, 7, 7, 6, 0, 6, 0, 5, 4, 0, 5, 6, 5, 4, 5, 4,
       5, 0, 1, 0, 6, 4, 6, 5, 6, 1, 1, 4, 5, 5, 1, 6, 5, 3, 1, 0, 3, 6,
       3, 5, 5, 2, 1, 6, 2, 2, 2, 3, 6, 1, 0, 0, 5, 2, 2, 1, 2, 1, 3, 5,
       6, 4, 1, 2, 1, 3, 3, 1, 0, 7, 7, 5, 7, 7, 7, 5, 7, 6, 4, 5, 0, 7,
       6, 7, 5, 7, 7, 6, 1, 1, 7, 7, 6, 5, 7, 7, 4, 5, 7, 5, 5, 7, 5, 5,
       5, 7, 7, 6, 0, 7, 1, 7, 6, 7, 6, 0, 7, 7, 7, 7, 4, 1, 2, 6, 7, 3,
       3])

In [95]:
test_labels

array([0, 0, 6, 0, 5, 2, 7, 2, 2, 7, 7, 5, 7, 7, 4, 4, 4, 2, 2, 6, 7, 7,
       7, 4, 6, 4, 3, 3, 6, 6, 5, 6, 4, 2, 4, 0, 4, 4, 2, 6, 6, 4, 0, 3,
       3, 3, 0, 0, 5, 2, 1, 5, 2, 1, 1, 1, 2, 4, 4, 1, 4, 3, 6, 4, 1, 6,
       4, 3, 3, 3, 0, 4, 4, 2, 6, 7, 7, 7, 5, 7, 6, 1, 5, 7, 7, 6, 6, 4,
       2, 7, 0, 0, 7, 1, 6, 6, 7, 4, 3, 3])

## CNN


### Build the feature extractor

In [96]:
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


In [97]:
feature_extractor = build_feature_extractor()

In [98]:
f1=feature_extractor.predict(X_for_classifier[1][1][None,:,:])

1/1 [==============================] - 1s 955ms/step


In [99]:
np.squeeze(f1).shape

(1024,)

In [100]:
plt.imshow(f1, cmap='gray')
plt.axis('off')
plt.show()

### Extract video features

In [101]:

def prepare_all_videos(videos, labels):
    num_samples = videos.shape[0] 

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, NB_FRAMES, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idv, video in enumerate(videos):
        # Extract features from the frames of the current video.
        for idf, frame in enumerate(video):
            frame_features[idv,idf, :] = feature_extractor.predict(frame[None,:,:])
            
    return frame_features, labels

### Train Feature extraction

In [102]:
#Compute the CNN feature map
#train_data,train_labels=prepare_all_videos(X_for_classifier,train_labels)

In [103]:
#save the model
#from numpy import savetxt
#filename = 'CNN_featuremap_train.npy'
#np.save(filename, train_data)

In [104]:
train_data = np.load("CNN_featuremap_train.npy")

In [105]:
train_data.shape

(177, 30, 1024)

### Test Feature extraction

In [106]:
#Compute the CNN feature map
#test_data,test_labels=prepare_all_videos(Xtest_for_classifier,test_labels)

In [107]:
#save the model
#from numpy import savetxt
#filename = 'CNN_featuremap_test.npy'
#np.save(filename, test_data)

In [108]:
test_data = np.load("CNN_featuremap_test.npy")

In [109]:
test_data.shape

(100, 30, 1024)

## Tranformer-based model

In [110]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask


In [111]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


### Utility functions for training

In [112]:
def get_compiled_model():
    sequence_length = 30
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    classes = 8

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        validation_split=0.15,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model


### Train the model:

In [113]:
trained_model = run_experiment()

Epoch 1/100
5/5 [==============================] - ETA: 0s - loss: 6.6106 - accuracy: 0.1400
Epoch 1: val_loss improved from inf to 4.98335, saving model to ./tmp\video_classifier
5/5 [==============================] - 1s 111ms/step - loss: 6.6106 - accuracy: 0.1400 - val_loss: 4.9834 - val_accuracy: 0.1111
Epoch 2/100
5/5 [==============================] - ETA: 0s - loss: 4.4084 - accuracy: 0.1600
Epoch 2: val_loss improved from 4.98335 to 2.97988, saving model to ./tmp\video_classifier
5/5 [==============================] - 0s 70ms/step - loss: 4.4084 - accuracy: 0.1600 - val_loss: 2.9799 - val_accuracy: 0.0741
Epoch 3/100
5/5 [==============================] - ETA: 0s - loss: 3.3063 - accuracy: 0.1533
Epoch 3: val_loss improved from 2.97988 to 2.43093, saving model to ./tmp\video_classifier
5/5 [==============================] - 0s 46ms/step - loss: 3.3063 - accuracy: 0.1533 - val_loss: 2.4309 - val_accuracy: 0.1481
Epoch 4/100
5/5 [==============================] - ETA: 0s - loss: 

Epoch 30/100
1/5 [=====>........................] - ETA: 0s - loss: 1.0099 - accuracy: 0.7500
Epoch 30: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 15ms/step - loss: 0.9014 - accuracy: 0.7267 - val_loss: 3.3834 - val_accuracy: 0.1852
Epoch 31/100
1/5 [=====>........................] - ETA: 0s - loss: 0.7675 - accuracy: 0.7188
Epoch 31: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 15ms/step - loss: 0.8742 - accuracy: 0.6867 - val_loss: 4.0121 - val_accuracy: 0.2593
Epoch 32/100
1/5 [=====>........................] - ETA: 0s - loss: 0.7446 - accuracy: 0.7500
Epoch 32: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 15ms/step - loss: 0.8219 - accuracy: 0.7200 - val_loss: 4.5339 - val_accuracy: 0.3333
Epoch 33/100
1/5 [=====>........................] - ETA: 0s - loss: 0.8568 - accuracy: 0.5938
Epoch 33: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 15ms/s

5/5 [==============================] - 0s 14ms/step - loss: 0.0374 - accuracy: 0.9800 - val_loss: 9.0505 - val_accuracy: 0.2593
Epoch 91/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0036 - accuracy: 1.0000
Epoch 91: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 14ms/step - loss: 0.0826 - accuracy: 0.9667 - val_loss: 8.3096 - val_accuracy: 0.1852
Epoch 92/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0970 - accuracy: 0.9688
Epoch 92: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 14ms/step - loss: 0.0879 - accuracy: 0.9800 - val_loss: 9.6937 - val_accuracy: 0.2593
Epoch 93/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0341 - accuracy: 1.0000
Epoch 93: val_loss did not improve from 1.64665
5/5 [==============================] - 0s 14ms/step - loss: 0.0431 - accuracy: 0.9867 - val_loss: 8.7164 - val_accuracy: 0.2963
Epoch 94/100
1/5 [=====>........................] - ETA: 0s - 

## Loading test data

In [117]:
 print(tf. __version__) 

2.9.0


In [114]:
videos_test, labels_test  = get_test_data()

In [115]:
#videos_test, labels_test=filter(filters,videos_test, labels_test)



In [116]:
builtx, builty= create_dataset(290,300, videos_test, labels_test)

NameError: name 'create_dataset' is not defined

In [ ]:
Xtest_for_classifier = np.array(builtx)
ytest_for_classifier = np.array(builty)

In [ ]:
ytest_for_classifier

In [ ]:
grayscale_batch=Xtest_for_classifier
rgb_batch = np.repeat(grayscale_batch[..., np.newaxis], 3, -1)

In [ ]:
ytest_for_classifier=func(ytest_for_classifier)

In [ ]:
ytest_for_classifier

In [ ]:
Xtest_for_classifier= rgb_batch

In [ ]:
Xtest_for_classifier.shape

In [ ]:
loss, accuracy = model.evaluate(Xtest_for_classifier, ytest_for_classifier)
print('Test accuracy :', accuracy)

In [ ]:
preds = model.predict(Xtest_for_classifier)
preds = np.argmax(preds, axis=1)
preds

In [ ]:
ytest_for_classifier

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
con_mat = confusion_matrix(ytest_for_classifier, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=con_mat)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']


In [ ]:
print(classification_report(ytest_for_classifier, preds,labels=[0,1]))